<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
%pip install tensorflow-gpu
clear_output()

In [ ]:
import os
from os.path import *
from pathlib import Path
from google.colab import drive
initialPath = realpath(Path.cwd())
print(initialPath)
drive.mount('/gdrive', force_remount=True)
os.chdir('/gdrive')
gdrivePath = realpath(Path.cwd())
mydrivePath = join(gdrivePath, 'My Drive')
os.chdir(mydrivePath)
from BashColors import C

In [ ]:
import tensorflow as tf
import timeit
logicalDevices = tf.config.list_logical_devices()

for dev in sorted(logicalDevices):
    print(dev)

print()
def cpu():
    cpu_device = tf.device('/device:CPU:0')
    
    with cpu_device:
        print('with: ', cpu_device)
    return cpu_device

def gpu():
    gpu_device = tf.device('/device:GPU:0')
    with gpu_device:
        print('gpu device')
    return gpu_device

cpu = cpu()
gpu = gpu()

with cpu:
    print('CPU')

with gpu:
    print('GPU')

In [ ]:

colabPath = join(mydrivePath, 'CoLab')
print(colabPath)

# Clone the entire repo.
if not exists(colabPath):
    print('Does not exist\n')
    with tf.device('/device:CPU:0'):
        print('exists')
        # !git clone -l -s git://github.com/iypc-team/CoLab.git CoLab  
else:
    print(f'{colabPath}{C.BGreen} already exists. OK') 
    os.chdir(colabPath)
    %ls


In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
print(device_name)
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')

In [ ]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth  

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
    random_image_cpu = tf.random_normal_initializer((100, 100, 100, 3))
    print(random_image_cpu)

with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random_normal_initializer((100, 100, 100, 3)) 
    print(random_image_gpu)

bs = '''
sess = tf.Session(config=config)

# Ensure that TF can detect the GPU.
try:
    sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
    print(
        '\n\nThis error most likely means that this notebook is not '
        'configured to use a GPU.  Change this in Notebook Settings via the '
        'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
    raise

'''
def cpu():
    tf.device('/cpu:0')
  
def gpu():
    sess.run(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

clear_output()

# Run the op several times.o
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()